In [2]:
# !pip install pandas
# !pip install pyyaml
# !pip install hpo3

In [3]:
import os
import pandas as pd
import numpy as np
from pyhpo import Ontology, HPOSet, Omim, stats, Gene
# import yaml

Ontology(os.path.join(os.getcwd(), 'phenotype', 'rawdl_20240310'))

In [4]:
# Declare the folder path for phenotype data source
phenotype_folder = os.path.join(os.getcwd(), 'phenotype', 'subset')

# Declare 7 Files
icd10omim = 'icd102omim_subset.tsv'
loinc2hpo = 'loinc2hpo_standardized.tsv'
omim2hpo = 'omim2hpo_subset.tsv'
orpha2omim = 'orpha2omim_subset.tsv'
orpha2hpo = 'orpha2hpo_subset.tsv'
snomed2hpo = 'snomed2hpo_subset.tsv'
snomed2orpha = 'snomed2orpha_subset.tsv'

# iderare yaml configuration file
yaml_file = 'iderare.yaml'

# Clinical data dummy in txt format separated with new line
clinical_data = 'clinical_data.txt'

# Read the clinical data and parse the data
icd10omim_df = pd.read_csv(os.path.join(phenotype_folder, icd10omim), sep='\t')
loinc2hpo_df = pd.read_csv(os.path.join(phenotype_folder, loinc2hpo), sep='\t')
omim2hpo_df = pd.read_csv(os.path.join(phenotype_folder, omim2hpo), sep='\t')
orpha2omim_df = pd.read_csv(os.path.join(phenotype_folder, orpha2omim), sep='\t')
orpha2hpo_df = pd.read_csv(os.path.join(phenotype_folder, orpha2hpo), sep='\t')
snomed2hpo_df = pd.read_csv(os.path.join(phenotype_folder, snomed2hpo), sep='\t')
snomed2orpha_df = pd.read_csv(os.path.join(phenotype_folder, snomed2orpha), sep='\t')

# Read line from clinical_data and parse the data to list
with open(clinical_data, 'r') as file:
    clinical_data_list = file.read().splitlines()

In [5]:
## Usage of HPO3
### Get Gene object
Gene.get('GBE1').hpo_set()
Omim.get(int('232200')).hpo_set()

HPOSet.from_serialized("7+93+97+105+295+660+787+822+823+939+991+1114+1402+1510+1538+1733+1892+1943+1997+2149+2240+2254+3077+3128+3162+3199+3593+4322+11463+12213")

In [6]:
# Convert SNOMED to ORPHA First
def snomed_orpha_parser(clinical_data, df):
    print('Trying to parse ORPHA from SNOMED-CT', clinical_data)
    
    if 'SNOMEDCT:' in clinical_data:
        if clinical_data not in df['code'].unique() :
            print('This SNOMED-CT code is not a clinical finding mapped with ORPHA, please check the SNOMED to ORPHA for diagnosis mapping.')
            snomed_sugg = df[df['code'].str.contains(clinical_data.strip('SNOMEDCT:'))]['code'].drop_duplicates()
            print('Sugggestion : It is possible that you mean any of this code:', (', ').join(snomed_sugg.values) , '?\n')
            return []
        else : 
            snomed_orpha = df[df['code'] == clinical_data]['orphanet_map'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(snomed_orpha), ' ORPHA code : ', (', ').join(snomed_orpha), '\n')
            return snomed_orpha
        
    else :
        print('The terminology is not recognized, please check if you have used correct terminology.')
        print('Allowable format : SNOMEDCT disorder semantic only SNOMEDCT:1212005', '\n')
        return []

In [7]:
# HPO Parser for Clinical Finding Related Terminology such as SNOMED, LOINC
def hpo_parser(clinical_data, df):    
    print('Trying to parse HPO from terminology', clinical_data)
    
    # LOINC case
    if 'LOINC:' in clinical_data:
        loinc_split = clinical_data.split('|')
        # Ensure that the forwarded data contain both LOINC and its interpretation
        if len(loinc_split) == 2:
            loinc = loinc_split[0]
            interpretation = loinc_split[1]
            
            # Handling if LOINC code not found in the database
            if loinc not in df['loinc'].unique() :
                print('LOINC data is not found in the database, please check the exact LOINC code.')
                loinc_sugg = df[df['loinc'].str.contains(loinc.strip('LOINC:'))]['loinc'].drop_duplicates()
                print('Did you mean any of this code:', (', ').join(loinc_sugg.values) , '?\n')
                return []
            
            loinc_hpo = df[df['loinc'] == loinc]

            # Handling if interpretation not suitable for the LOINC code
            if interpretation not in loinc_hpo['interpretation'].unique() :
                print('Interpretation is invalid, please check if you have used correct interpretation.')
                interpretation_sugg = df[df['loinc']==loinc]['interpretation'].drop_duplicates()
                print('Available interpretation for code', loinc, ' : ', (' or ').join(interpretation_sugg.values) , '\n')
                return []
            
            loinc_hpo = loinc_hpo[loinc_hpo['interpretation'] == interpretation]['hpoTermId'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', loinc_hpo, '\n')
            return loinc_hpo
        else:
            print('LOINC data is missing either code / interpretation.')
            print('Example : LOINC:721-1|H for Qn lab examination OR LOINC:721-1|NEG for Nominal / Ordinal Lab Examination', '\n')
            return []
        
    # SNOMED-CT case
    elif 'SNOMEDCT:' in clinical_data:
        if clinical_data not in df['SNOMED_CT_ID'].unique() :
            print('This SNOMED-CT code is not a clinical finding mapped with HPO, please check the SNOMED to OMIM for diagnosis mapping.')
            snomed_sugg = df[df['SNOMED_CT_ID'].str.contains(clinical_data.strip('SNOMEDCT:'))]['SNOMED_CT_ID'].drop_duplicates()
            print('Sugggestion : It is possible that you mean any of this code:', (', ').join(snomed_sugg.values) , '?\n')
            return []
        else : 
            snomed_hpo = df[df['SNOMED_CT_ID'] == clinical_data]['HPO_ID'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(snomed_hpo), ' HPO code : ', (', ').join(snomed_hpo), '\n')
            return snomed_hpo
        
    # Not recognized case
    else:
        print('The terminology is not recognized, please check if you have used correct terminology.')
        print('Example : LOINC:2862-1|L for Qn lab examination OR LOINC:725-2|NEG for categoric lab examination of SNOMEDCT:48610005 for Clinical Finding', '\n')
        return []

In [8]:
# OMIM Parser used for diagnosis related terminology ICD-10, ORPHA, SNOMED-CT to be translated to OMIM
def omim_parser(clinical_data,df):
    print('Trying to parse OMIM from terminology', clinical_data)

    # ICD-10 case
    if 'ICD-10:' in clinical_data:
        if clinical_data not in df['ICD10'].unique() :
            print('ICD-10 data is not found in the database, please check the exact ICD-10 code.')
            icd_sugg = df[df['ICD10'].str.contains(clinical_data.strip('ICD-10:'))]['ICD10'].drop_duplicates()
            print('Did you mean any of this code:', (', ').join(icd_sugg.values) , '?\n')
            return []
        else : 
            icd_omim = df[df['ICD10'] == clinical_data]['OMIM'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(icd_omim), ' OMIM code : ', (', ').join(icd_omim), '\n')
            return icd_omim
        
    # ORPHA case
    elif 'ORPHA:' in clinical_data:
        if clinical_data not in df['ORPHA'].unique() :
            print('ORPHA data is not found in the database, please check the exact ORPHA code.')
            orpha_sugg = df[df['ORPHA'].str.contains(clinical_data.strip('ORPHA:'))]['ORPHA'].drop_duplicates()
            print('Did you mean any of this code:', (', ').join(orpha_sugg.values) , '?\n')
            return []
        else : 
            orpha_hpo = df[df['ORPHA'] == clinical_data]['OMIM'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(orpha_hpo), ' OMIM code : ', (', ').join(orpha_hpo), '\n')
            return orpha_hpo
    
    else:
        print('The terminology is not recognized, please check the input format.')
        print('Allowable format is : ICD-10:xxxx OR ORPHA:xxxxx for clinical disorder', '\n')
        return []

In [18]:
phenotypes = [
      "HP:0002366",
      "HP:0005561",
      "HP:0001903",
      "HP:0010972",
      "HP:0001541",
      "HP:0004333",
      "HP:0001396",
      "HP:0002910",
      "HP:0001531",
      "HP:0001399",
      "HP:0001433",
      "HP:0003073",
      "HP:0003233",
      "HP:0006568",
      "HP:0002151",
      "HP:0200114",
      "HP:0001653",
      "HP:0000938",
      "HP:0001873",
    ]

for p in phenotypes : 
    # print(p, snomed2hpo_df[snomed2hpo_df['HPO_ID'] == p]['SNOMED_CT_ID'].to_list(), snomed2hpo_df[snomed2hpo_df['HPO_ID'] == p]['HPO_NAME'].to_list())
    snomed_list = snomed2hpo_df[snomed2hpo_df['HPO_ID'] == p]['SNOMED_CT_ID'].to_list()
    hpo_name = snomed2hpo_df[snomed2hpo_df['HPO_ID'] == p]['HPO_NAME'].to_list()
    if len(snomed_list) == 0 :
        print(p, 'is not found in the database')
    for i in range(len(snomed_list)) : 
        print(p, snomed_list[i], hpo_name)

HP:0002366 is not found in the database
HP:0005561 SNOMEDCT:127035006 ['Abnormal bone marrow cell morphology']
HP:0001903 SNOMEDCT:165397008 ['Anemia', 'Anemia']
HP:0001903 SNOMEDCT:271737000 ['Anemia', 'Anemia']
HP:0010972 SNOMEDCT:70730006 ['Anemia of inadequate production']
HP:0001541 SNOMEDCT:389026000 ['Ascites']
HP:0004333 is not found in the database
HP:0001396 SNOMEDCT:197446008 ['Cholestasis', 'Cholestasis', 'Cholestasis']
HP:0001396 SNOMEDCT:30144000 ['Cholestasis', 'Cholestasis', 'Cholestasis']
HP:0001396 SNOMEDCT:33688009 ['Cholestasis', 'Cholestasis', 'Cholestasis']
HP:0002910 SNOMEDCT:166603001 ['Elevated circulating hepatic transaminase concentration', 'Elevated circulating hepatic transaminase concentration', 'Elevated circulating hepatic transaminase concentration']
HP:0002910 SNOMEDCT:166643006 ['Elevated circulating hepatic transaminase concentration', 'Elevated circulating hepatic transaminase concentration', 'Elevated circulating hepatic transaminase concentration'

In [10]:
# Check the dataframe
icd10omim_df['ICD10'].value_counts()
orpha2omim_df['ORPHA'].value_counts()
snomed2orpha_df['code'].value_counts()
snomed2hpo_df['SNOMED_CT_ID'].value_counts()

SNOMEDCT:127026004    3
SNOMEDCT:42658009     3
SNOMEDCT:30721006     3
SNOMEDCT:64779008     3
SNOMEDCT:278849000    3
                     ..
SNOMEDCT:254044004    1
SNOMEDCT:238090007    1
SNOMEDCT:78494001     1
SNOMEDCT:122481008    1
SNOMEDCT:255055008    1
Name: SNOMED_CT_ID, Length: 4401, dtype: int64